In [26]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import torch

In [33]:
pt_model = YOLO("./runs/detect/train8/weights/best.pt")
ov_model = YOLO("./runs/detect/train8/weights/best_openvino_model")

# Warmup
dummy_frame = torch.rand(1, 3, 704, 704)
ov_model.predict(dummy_frame)
pt_model.predict(dummy_frame)

print('Warmed up')

Loading ./runs/detect/train8/weights/best_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference on (CPU)...

0: 704x704 (no detections), 129.6ms
Speed: 0.0ms preprocess, 129.6ms inference, 4.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 (no detections), 128.0ms
Speed: 0.0ms preprocess, 128.0ms inference, 3.2ms postprocess per image at shape (1, 3, 704, 704)
Warmed up


In [39]:
vid_cap = cv2.VideoCapture('./data/Movie_009.mp4')

start_t = time.time()

prev_frame_t = 0
curr_frame_t = 0
total_frames = 0

while True:
    curr_frame_t = time.time()
    ret, frame = vid_cap.read()
    
    if not ret:
        break
    
    results = ov_model.predict(frame, verbose=False)
    annotated_frame = results[0].plot()
    
    fps = int(1/(curr_frame_t - prev_frame_t))
    cv2.putText(annotated_frame, f'FPS {fps}', (25, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('ov_model window', annotated_frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
    
    prev_frame_t = curr_frame_t
    total_frames += 1
    if total_frames >= 250:
        break
    
    
end_t = time.time()
time_taken = end_t - start_t
print(f'time taken: {time_taken} sec, fps: {int(total_frames / time_taken)}')

vid_cap.release()
cv2.destroyAllWindows()

time taken: 9.740906953811646 sec, fps: 25


In [23]:
vid_cap = cv2.VideoCapture('./data/Movie_009.mp4')
vid_cap.get(cv2.CAP_PROP_FPS)

30.0

In [3]:
cv2.destroyAllWindows()